In [ ]:
def Previsaodotempo(Tmax,Tmin,Umi):
    if Tmax > Tmin:
        return True
    else:
        return False

In [ ]:
import tkinter as Tkinter
import random
 
class minhaApp_tk(Tkinter.Tk):
# Tkinter.tk é a classe base para a janela padrão. A nossa classe minhaApp_tk irá herdar todas as funcionalidades da classe padrão.
 
    def __init__(self,parent):
        Tkinter.Tk.__init__(self,parent)
        # no construtor da nossa classe, apenas chamamo o construtor da classe pai, Tkinter.Tk.__init__()).
        self.parent = parent
        # geralmente necessitaremos  de acessar o pai de um objeto. É uma boa técnica  sempre salvar uma referencia ao pai.
        self.initialize()
        #no método initialize criamos os demais objetos que serão apresentados na tela, inicializamos as variáveis globais (irc..),
                   #inicializamos o hardware caso necessário, etc..
         
         
    def initialize(self):     #aqui criamos os widgets que serão apresentados na tela
        self.grid()     #vamos dispor os objetos na tela dentro de uma grade         
        
        self.instruLabel = Tkinter.Label(self,text="Insira os dados abaixo para descobrir se choverá amanhã", height = 2, font=("Helvetica", 10))
        self.instruLabel.grid(columnspan=3,column=0,row=0,sticky='EW')  #deixamos na posição correspondente à terceira coluna, segunda linha

        
        self.lblC= Tkinter.Label(self, text="Temperatura Maxima")     #este objeto irá apresentar o primeiro número
        self.lblC.grid(column=0, row=1)               #nesta posição
         
        self.lblSep= Tkinter.Label(self, text="Temperatura Mínima")     #este objeto irá apresentar o segundo  número
        self.lblSep.grid(column=1, row=1)               #nesta posição     
 
 
        self.lblF= Tkinter.Label(self, text="Umidade Média")     #este objeto irá apresentar o terceiro número
        self.lblF.grid(column=2, row=1)               #nesta posição       
 
        self.entrymax = Tkinter.Entry(self)               #criamos o campo de texto onde está disponível o valor em gras centígrados
        self.entrymax.grid(column=0,row=2,sticky='EW')    #deixamos na posição correspondente a primeira coluna, segunda linha  
        
        self.entrymin = Tkinter.Entry(self)               #criamos o campo de texto onde está disponível o valor em gras centígrados
        self.entrymin.grid(column=1,row=2,sticky='EW')    #deixamos na posição correspondente a primeira coluna, segunda linha  
 
        self.entryumi = Tkinter.Entry(self)             #criamos o campo de texto onde esta disponivel o valor em graus fahrenheit
        self.entryumi.grid(column=2,row=2,sticky='EW')  #deixamos na posição correspondente à terceira coluna, segunda linha
 
 
 
        self.btnCalculaF = Tkinter.Button(self,text=u"Prever o tempo",command=self.OnButtonCalculaFClick)  #criamos o objeto botão
                                                                    #associamos o clicar do botão ao método OnButtonCalculaFClick
        self.btnCalculaF.grid(column=1,row=3) 
        
        self.resuLabel = Tkinter.Label(self,text=" ", height = 2, font=("Helvetica", 28))
        self.resuLabel.grid(columnspan=3,column=0,row=5,sticky='EW')  #deixamos na posição correspondente à terceira coluna, segunda linha

    def OnButtonCalculaFClick(self):     #calcula o valor em fahrenheit e mostra no campo correto
        Tmax= float(self.entrymax.get())  #le o valor no campo de entrada de centígrados
        Tmin= float(self.entrymin.get())  #le o valor no campo de entrada de centígrados
        Umi= float(self.entryumi.get())  #le o valor no campo de entrada de centígrados
        resultado = Previsaodotempo(Tmax,Tmin,Umi)
        if resultado == True:
            resultado = 'VAI CHOVER'
        else:
            resultado = 'NÃO VAI CHOVER'
        self.resuLabel.config(text=resultado)
 
#este é ponto onde o programa se inicia
#se o programa foi chamado a partir do interpretador python, o _name_  automaticamente será "__main__"
 
if __name__ == "__main__":
    app = minhaApp_tk(None)      #criamos uma aplicação sem nenhum pai, pois é a principal.
    app.title('Choverá amanha?')    #especificamos o título de nossa aplicação
    app.mainloop()                  #o programa entra no loop de espera de eventos (pressionar de menus, botões, etc..)

Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\marco\Anaconda3\lib\tkinter\__init__.py", line 1699, in __call__
    return self.func(*args)
  File "<ipython-input-1-bba4ecb5be69>", line 56, in OnButtonCalculaFClick
    resultado = Previsaodotempo(Tmax,Tmin,Umi)
NameError: name 'Previsaodotempo' is not defined
